In [118]:
!pip install git+git://github.com/geopandas/geopandas.git

  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-afqliznz
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-afqliznz
  Created wheel for geopandas: filename=geopandas-0.8.0+72.ge506de5-py2.py3-none-any.whl size=973052 sha256=0e965bd7f661bd643786e116df0c1e019f647d32cf3322809841a5046710dea1
  Stored in directory: /tmp/pip-ephem-wheel-cache-s1jrawgd/wheels/91/24/71/376c9c67192694168352afcccc2d264248f7e2cc6192997186
Successfully built geopandas


In [119]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving coronavirus-commercants-parisiens-livraison-a-domicile.geojson to coronavirus-commercants-parisiens-livraison-a-domicile (2).geojson
Saving logement-encadrement-des-loyers.geojson to logement-encadrement-des-loyers (2).geojson
Saving logements-sociaux-finances-a-paris.geojson to logements-sociaux-finances-a-paris (2).geojson
Saving quartier_paris.geojson to quartier_paris (2).geojson
User uploaded file "coronavirus-commercants-parisiens-livraison-a-domicile.geojson" with length 1391320 bytes
User uploaded file "logement-encadrement-des-loyers.geojson" with length 11711111 bytes
User uploaded file "logements-sociaux-finances-a-paris.geojson" with length 1985756 bytes
User uploaded file "quartier_paris.geojson" with length 360282 bytes


In [120]:
import pandas as pd
import geopandas as gpd
import json
import io


In [121]:
#importation des données des logements sociaux à Paris
file_name='logements-sociaux-finances-a-paris.geojson'
file = open(file_name)
gdf_logements_soc = gpd.read_file(file)

#suppression de variables
gdf_logements_soc.drop(['nature_programme','arrdt','ville','n_livraison','adresse_programme','code_postal','annee','bs','nb_plai','nb_plus','nb_pluscd','nb_pls','mode_real','commentaires','coord_x_l93','coord_y_l93'], 1, inplace=True)

# on importe les données
file_name='quartier_paris.geojson'
file = open(file_name)
gdf = gpd.read_file(file)

def quartierappartenance(point):
    for i in range(80):
        if point.within(gdf["geometry"][i]):
            return gdf["l_qu"][i]

def numquartierappartenance(point):
    for i in range(80):
        if point.within(gdf["geometry"][i]):
            return gdf["c_qu"][i]


gdf_logements_soc["quartier"]=gdf_logements_soc["geometry"].apply(quartierappartenance)
gdf_logements_soc["numquartier"]=gdf_logements_soc["geometry"].apply(numquartierappartenance)

# on va indiquer le nombre de logement qu'on a par quartier administratif

gdf_logements_sociaux = gdf_logements_sociaux.groupby(['numquartier','quartier']).sum()
gdf_logements_sociaux

,,nb_logmt_total
numquartier,quartier,
1,Saint-Germain-l'Auxerrois,215
2,Halles,445
3,Palais-Royal,137
4,Place-Vendôme,12
5,Gaillon,5
...,...,...
76,Combat,1026
77,Belleville,1542
78,Saint-Fargeau,2721


In [122]:
# on importe les données
file_name='logement-encadrement-des-loyers.geojson'
file = open(file_name)
gdf_l = gpd.read_file(file)


# suppression de variables
gdf_l.drop(['ville','epoque','meuble_txt','code_grand_quartier','piece','max','min','id_zone','annee'],1,inplace=True)

# on va calculer la moyenne des loyers dans chaque quartier administratif
gdf_loyer=gdf_l.groupby(['nom_quartier','id_quartier']).mean()
gdf_loyer.sort_values(by='id_quartier')

,,ref
nom_quartier,id_quartier,
St-Germain-l'Auxerrois,1,28.231250
Halles,2,25.243750
Palais-Royal,3,28.231250
Place-Vendôme,4,28.231250
Gaillon,5,28.231250
...,...,...
Combat,76,22.700000
Belleville,77,23.978125
Saint-Fargeau,78,20.703125


In [141]:
# on importe les données sur les commerces parisiens
file_name='coronavirus-commercants-parisiens-livraison-a-domicile.geojson'
file = open(file_name)
gdf_co = gpd.read_file(file)

gdf_co['quartier']=0

# on va indiquer à quel quartier administratif appartient le commerce 
gdf_co["quartier"]=gdf_co["geometry"].apply(quartierappartenance)
gdf_co['numquartier']=gdf_co['geometry'].apply(numquartierappartenance)
# on va supprimer des variables 
gdf_co.drop(['code_postal','description','nom_du_commerce','adresse','telephone','services','mail','type_de_commerce','site_internet','precisions'],1,inplace=True)

# on va créer une variable qui va indiquer le nombre total de commerce par quartier administratif
gdf_co['nombre_total_de_commerce']=1
gdf_commerce = gdf_co.groupby(['numquartier','quartier']).sum()